# Stange's code

In [16]:
#########################################
# Toy implementation of factoring algorithm (Algorithm 2.2) in 
# "Factoring using multiplicative relations modulo n:
#      a subexponential factoring algorithm inspired by the index calculus
# by Katherine E. Stange
#########################################

def get_order(n, g, B=10, rels=10, verbose=True, alphas=False, texify=False):
    # INPUT:
    # n = modulus
    # g = base
    # B = bound for primes
    # rels = number of additional relations
    # verbose = whether to print info as you go
    # alphas = whether to return the alpha values
    # textify = whether to print latex for algorithm
    # OUTPUT:  order of g modulo n (with high probability)
    
    # Set up factor base
    if verbose:
        print("Attempting to compute the order of ", g, "modulo", n)
    numPrimes = prime_pi(B)  # the number of primes <= B
    if verbose:
        print("The number of primes in the factor base:", numPrimes)
        print("These are:", [nth_prime(m) for m in range(1, numPrimes+1)])
    
    # Set the number of relations to find
    relsDesired = numPrimes + rels  # the number of relations to find
    if verbose:
        print("We are looking for ", relsDesired, "relations.")
    vectors = []  # to store the relations we find
        
    # Main relation finding loop
    searchCount = 0 # number of smoothness tests run
    relsFound = 0 # number of relations found
    while( relsFound < relsDesired ):
        
        # take a random power of x
        
        x = randint(1,n)        
        residue = ZZ(Mod(g,n)^x)
        
        # trial division to test smoothness
        fac = [0 for _ in range(B+1)]
        for p in primes(B):
            while Mod(residue,p) == 0:
                residue = residue/p
                fac[p] += 1
        if residue == 1:  # store a relation if smooth
            if verbose:
                print("found a relation:", g, "^", x, "is", factor(ZZ(Mod(g,n)^x)))
            if texify:
                print(g,"^{",x,"}&=",latex(factor(ZZ(Mod(g,n)^x))),",\\\\")
            vec = vector([fac[nth_prime(m)] for m in range(1,numPrimes+1)])
            vectors.append([vec,x])
            relsFound += 1
            
        searchCount += 1
        
    if verbose:
        print("I searched through ", searchCount, "powers of ", g)
        
    # Linear algebra phase
    relMatrix = matrix([vectors[i][0] for i in range(relsDesired)]).transpose()
    if verbose:
        print("The relation matrix is (cols are relations):")
        print(relMatrix)
    
    if texify:
        print(latex(relMatrix))
    kernel = relMatrix.right_kernel().basis()
        
    if verbose:
        print("The right kernel has basis:")
        print(kernel)
    if texify:
        print(latex(matrix(kernel)))
    alphaVals = []
    for basisVec in kernel:
        alpha = sum([basisVec[i]*vectors[i][1] for i in range(relsDesired)])
        alphaVals.append(alpha)
    if verbose:
        print("The corresponding sums of the x's are:")
        print(alphaVals)
    if texify:
        print(latex(alphaVals))
        
    # GCD phase
    finalGCD = 0
    for alpha in alphaVals:
        finalGCD = gcd(finalGCD,alpha)
        
    # Report and return
    if verbose:
        print("Their gcd is:", finalGCD)
        print("******* Relation to reality? ******")
        print("Sage's factorization of", n, "is", factor(n))
        print("The actual multiplicative order is:", Mod(g,n).multiplicative_order())
        if Mod(g,n).multiplicative_order() == finalGCD:
            print("Success!")
        else:
            print("Failure!")
            print("The ratio of the expected value to the real value is (1=success):", finalGCD/Mod(g,n).multiplicative_order())
    if alphas:
        return(finalGCD,[a/finalGCD for a in alphaVals])
    return(finalGCD)
    

In [17]:
n=20191
g=4
B=10
c=10

get_order(n, g, B, c, verbose=False, alphas=True)

(30, [488, 25, 396, 179, 473, 401, 796, 342, 178, 545, 276, 364, 447])

# My code

In [110]:
import numpy as np
def factorisation_problem(n,g,B=10,c=10):    
    # n = composite modulus
    # g = base g<n
    # B = bound for primes
    # c = number of additional relations
    # multp = integer used to compute non trivial factors
    # OUTPUT:  factors of n
    
    if gcd(g,n)!=1:      # g must be a unit for the algoritmh to make sense     # write about this
        factor1=gcd(g,n)
        print(f"{g} is a unit modulus {n}. This means that gcd({g},{n}) is a nontrivial factor of {n}: gcd({g},{n}):",gcd(g,n))
        factor2=n/gcd(g,n)
        print(f"{n}={factor1}*{factor2}")
        return (factor1,factor2)
    
    numb_primes=prime_pi(B) # number of primes below B    

    #-------------------phase 1: Relation finding------------------------- 
    relsFound = 0               # number of relations found
    relsDesired=numb_primes+c   # number of relations desired
    rand_x=[]                             # x vector
    F=matrix(numb_primes,numb_primes+c)   # relation Matrix
#     print(f"{relsFound}/{relsDesired} relations found")
    while( relsFound < relsDesired ):
        
        x = randint(1,n)

        residue = ZZ(Mod(g,n)^x)
        f=[]

        #-------------------Attempt factorisation of g^x------------------------------
        for p in primes(B):
            exponent=0
            while Mod(residue,p)==0:
                residue=residue/p           
                exponent+=1
            f.append(exponent)

        #------------------------------Append the vector if it is B-smooth---------------------
        if residue==1:
            f=vector(f)
            rand_x.append(x)
            F[:,relsFound]=f
            relsFound+=1
#             print(f"{relsFound}/{relsDesired} relations found")

    
    #------------------------------phase 2: Linear Algebra---------------------------------------
 
    kernel_basis=F.right_kernel().basis() # obtain the basis
    
        #-----------------------Find alphas--------------------------------
    alpha=[]
    for base_vec in kernel_basis:
        alp=np.dot(base_vec,rand_x)  
        alpha.append(alp)  #list of all the alphas
    #----------------------phase 3: Computing order of g mod n----------------------------
    
#     print(f"the alphas are {alpha}")
    order=Integer(gcd(alpha)) # probabilistic 
    print(f"the order is {order}")
    if order==n-1:
        print(f"{n} is prime since there exist {g} mod {n} such that the order of {g} is {n-1}") # write about this
        return n,1  
    
    print(f"the value of {g}^{order} mod {n} is",Mod(g^order,n))
    if Mod(g^order,n)!=1:
        print("Error: the order is incorrect")
        return "Error"
    #---------------------------------------Factorising n------------------------------------
    
    # this is to be proved to work for any order r st p|r
    # we know it works for Shor's algorithm so you can prob extend the proof
#     m=multp*order #find highest power of 2 that divides 'multp' 
#     exponent=0
    for p in primes(B):      # could instead make a recursion here to get the prime divisors of order
        if order%p==0 and g^(order/p)+1%n!=0:  # the second case gives a trivial factorisation
            factor1=gcd(g^(order/p)+1,n)                 # prove this whole thing

            if factor1==1 or factor1==n:
                print(p)
                factor1=gcd(g^(order/p)-1,n)            # doesnt always work
            factor2=n/factor1
            print(f"the factors of {n} are {factor1} and {factor2}: {factor1*factor2}={factor1}*{factor2}")  
            return factor1,factor2,order


In [100]:
n=781
g=11
B=10

factorisation_problem(n,g,B)   # add some failsafes and test for bunch of numbers


11 is a unit modulus 781. This means that gcd(11,781) is a nontrivial factor of 781: gcd(11,781): 11
781=11*71


(11, 71)

In [101]:
#test for large primes
    
g=11
B=10    
for n in [6779,6781,6791,5483,5501 ,5503 ,5507,7853,7867,7873]:
    factorisation_problem(n,g,B)    # this may outut 1*n if n is prime
# figure out what happens in this case


the order is 6778
6779 is prime since there exist 11 mod 6779 such that the order of 11 is 6778
the order is 3390
the value of 11^3390 mod 6781 is 1
the factors of 6781 are 6781 and 1: 6781=6781*1
the order is 6790
6791 is prime since there exist 11 mod 6791 such that the order of 11 is 6790
the order is 5482
5483 is prime since there exist 11 mod 5483 such that the order of 11 is 5482
the order is 2750
the value of 11^2750 mod 5501 is 1
the factors of 5501 are 5501 and 1: 5501=5501*1
the order is 5502
5503 is prime since there exist 11 mod 5503 such that the order of 11 is 5502
the order is 2753
the value of 11^2753 mod 5507 is 1
the order is 7852
7853 is prime since there exist 11 mod 7853 such that the order of 11 is 7852
the order is 3933
the value of 11^3933 mod 7867 is 1
the factors of 7867 are 1 and 7867: 7867=1*7867
the order is 2624
the value of 11^2624 mod 7873 is 1
the factors of 7873 are 7873 and 1: 7873=7873*1


In [111]:
# test for odd order

# 64 mod 283 has order 47
# 47 is prime so can't quite do much with that 

# 2 mod 152942113 has order 4247705 

n=152942113
g=2
factorisation_problem(n,g)

the order is 4247705
the value of 2^4247705 mod 152942113 is 1
5
the factors of 152942113 are 12343 and 12391: 152942113=12343*12391


(12343, 12391, 4247705)